In [ ]:
# !pip install transformers
# !pip install nlpaug
# !git clone https://github.com/joseph1723/CS376_Final_Project.git

In [10]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification, BertModel

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2, output_attentions=True)

if torch.cuda.is_available() :
  print("CUDA")
  device = torch.device("cuda")
  model.to(device)
else :
  print("CPU")
  device = torch.device("cpu")
  model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

CPU


In [12]:
class TestDataset(Dataset) :
  #Dataset - English/typo-added/labeled
  def __init__(self, df) :
    self.df = df
  
  def __len__(self) :
    return len(self.df)
  
  def __getitem__(self, idx):
    text = self.df.iloc[idx, 0]
    label = self.df.iloc[idx, 1]
    return text, label

Hyperparameters

In [13]:
train_rate, test_rate = 0.5, 0.05
itr = 1
p_itr = 100
epochs = 5
batch = 10

In [14]:
total_df = pd.read_csv('augmented_data/Dataset_aug_complex_6513_.csv', sep=',')
total_df.dropna(inplace=True)
total_df = total_df[["text", "label"]]
total_df["label"] = [1 if i == "nothate" else 0 for i in total_df["label"]]
print(total_df)
total_dataset = TestDataset(total_df)
total_loader = DataLoader(total_dataset, batch_size=batch, shuffle=True)
max_len = 0

for text, _ in total_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    l = max(len(e) for e in encoded_list)
    max_len = l if l>max_len else max_len

print(max_len)

                                                    text  label
0      Ellen Page has come out as a man according to ...      1
1      Ellen Page has come out as a meat according to...      1
2      Ellen Page has come out as a chowder according...      1
3      Ell$B (aYe has come out as a man accoTRinr to ...      1
4      Ellen Page has come out as a man according to ...      1
...                                                  ...    ...
39073                 Being b1tch is unnatural and wrong      0
39074                   Being ass is unnatural and wrong      0
39075                   hsing gay is unnatural and wrigg      0
39076                   Eein9 gay is unnatural and wrun9      0
39077                   Begjg gay is Dnnat&r*l and wrong      0

[39078 rows x 2 columns]
163


In [15]:
#Train Set
test_df, train_df, _ = np.split(total_df, [int(test_rate*len(total_df)), int(test_rate*len(total_df) + train_rate*len(total_df))])
print(len(test_df), len(train_df))
train_dataset = TestDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=True)

1953 19539


In [16]:
optimizer = Adam(model.parameters(), lr=1e-6)

total_loss = 0
total_len = 0
total_correct = 0
X=1

model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        MAX_LEN = max(len(e) for e in encoded_list)
        padded_list =  [e + [0] * (MAX_LEN-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels, return_dict=False)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=X)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0
        itr+=1

/var/folders/kx/qy_c0smd4c14rc316r9rcq3m0000gn/T/ipykernel_70493/1009792008.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(label)


ValueError: too many values to unpack (expected 2)

In [ ]:
model.eval()

total_loss = 0
total_len = 0
total_correct = 0

for text, label in test_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    MAX_LEN = max(len(e) for e in encoded_list)
    padded_list =  [e + [0] * (MAX_LEN-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels, return_dict=False)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)

In [ ]:
class neuNet(nn.Module):
    def __init__(self):
        super(neuNet,self).__init__()
        ## This is the number of hidden nodes in each layer (512)
        ## This is the linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(163, 768)
        ## This is also linear layer but (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(768,1)
        #The dropout layer (p=0.2)
        self.dropout = nn.Dropout(0.3)
    
    def forward(self,x):
        x = x.view(-1,768)
        x = F.relu(self.fc1(x))
        x = self.droput(x)
        x = self.fc2(x)
        return x

MLP_model = neuNet()

In [ ]:
class Feed_Forward_network(nn.Module):
  def __init__(self, dh, dff):
    super(Feed_Forward_network, self).__init__()
    self.linear1 = nn.Linear(dh, 1)
    # self.linear2 = nn.Linear(dff,1)
    self.final = nn.Linear(dh, 2)
  def forward(self, x, y, valid_length, word_index):
    #(Batch, Max_len, dh/emb_dim)
    out1 = self.linear1(x)
    out1 = F.relu(out1)
    print("out1: ", out1.shape)
    out_soft = masked_softmax(out1.transpose(1,2), valid_length)
    print("out_soft: ", out_soft.shape)
    out_weighted= x.mul(out_soft.transpose(1,2))
    print("out_weighted: ", out_weighted.shape)
    weighted_sum = torch.sum(out_weighted, axis=1)
    print("weighted_sum: ", weighted_sum.shape)
    logits = self.final(weighted_sum)
    fil_tgt = 1
    loss = F.nll_loss(F.log_softmax(logits,dim=-1), y, ignore_index = 0, reduction = 'none')
    loss = loss.sum().mean()
    preds = (torch.softmax(logits, dim=-1)).argmax(dim=-1)
    return preds, fil_tgt, loss